In [20]:
a = torch.Tensor(((-0.2,0.2,-0.3), (-0.2,0.2,-0.3)))
b = torch.Tensor(((-0.1,0.2,-0.3), (-0.2,0.2,-0.3)))
a, b

(tensor([[-0.2000,  0.2000, -0.3000],
         [-0.2000,  0.2000, -0.3000]]),
 tensor([[-0.1000,  0.2000, -0.3000],
         [-0.2000,  0.2000, -0.3000]]))

In [21]:
F.binary_cross_entropy(a,b)

RuntimeError: all elements of input should be between 0 and 1

In [66]:
for data in train_loader:
    print(data)

tensor([[[-0.0102, -0.0041, -0.0140,  ..., -0.0046, -0.0065, -0.0044]],

        [[ 0.0037,  0.0034,  0.0044,  ..., -0.0039, -0.0050, -0.0062]],

        [[ 0.0096,  0.0185,  0.0114,  ...,  0.0207,  0.0199,  0.0183]],

        ...,

        [[ 0.0428,  0.0116,  0.0892,  ..., -0.0258, -0.0315, -0.0325]],

        [[-0.0065, -0.0063, -0.0056,  ...,  0.0004, -0.0003,  0.0035]],

        [[-0.0656, -0.0687, -0.0663,  ..., -0.0652, -0.0806, -0.0772]]])
tensor([[[-3.5675e-02, -1.9531e-03, -1.2939e-02,  ...,  6.0431e-01,
           5.6027e-01,  3.8354e-01]],

        [[ 3.4119e-02,  3.5278e-02,  3.7506e-02,  ..., -1.8616e-03,
           1.8311e-04,  1.6174e-03]],

        [[-9.7046e-03, -9.9792e-03, -8.9417e-03,  ...,  1.0468e-02,
           9.0332e-03,  5.3101e-03]],

        ...,

        [[ 1.0803e-02, -2.3071e-02,  2.1027e-02,  ...,  1.2329e-02,
          -2.7832e-02,  1.5869e-02]],

        [[-1.5259e-03, -2.1973e-03, -2.0142e-03,  ...,  5.8899e-03,
           2.7679e-02, -3.7537e-03]],


tensor([[[-2.1667e-03,  2.1545e-02, -4.3030e-03,  ...,  9.2468e-03,
          -3.9368e-03, -2.4414e-04]],

        [[ 3.1769e-02,  2.9938e-02,  2.5909e-02,  ...,  3.0823e-03,
           6.4087e-03,  3.3569e-03]],

        [[-1.2817e-03,  7.8125e-03, -3.0518e-05,  ...,  6.6528e-03,
           1.1597e-03,  3.3264e-03]],

        ...,

        [[-1.3611e-02, -7.9956e-03,  2.0447e-03,  ..., -8.3923e-03,
           1.4740e-02, -1.7975e-02]],

        [[ 3.9368e-02,  3.6041e-02,  3.4729e-02,  ...,  1.7151e-02,
          -3.8147e-02, -8.8501e-04]],

        [[ 1.8616e-03,  1.5869e-03,  2.0752e-03,  ...,  5.8289e-03,
           1.3733e-03,  0.0000e+00]]])
tensor([[[ 2.2583e-03,  3.2043e-03,  3.0518e-03,  ...,  1.0071e-03,
           9.1553e-05,  0.0000e+00]],

        [[ 6.7139e-03, -6.8970e-03,  5.1270e-03,  ...,  7.9956e-03,
          -8.7585e-03, -5.8289e-03]],

        [[ 1.1780e-02,  1.0651e-02,  9.7961e-03,  ...,  8.1787e-03,
           6.4087e-03,  4.3640e-03]],

        ...,

        [

In [68]:
F.binary_cross_entropy(data[0], data[0])

RuntimeError: all elements of input should be between 0 and 1

In [67]:
data.shape

torch.Size([100, 1, 16000])

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
import glob
import librosa
import numpy as np

In [2]:
class AudioDataset(Dataset):
    def __init__(self, path, transform=None, sample_rate=16000):
        self.path = path
        self.data_list = glob.glob(self.path + '/*.wav')
        
        self.transform = transform
        self.sr = sample_rate
        
    def __len__(self):
        return len(self.data_list)
    
    def __getitem__(self, idx):
        data_path = self.data_list[idx]
        data, sr = librosa.load(data_path, sr=self.sr)
        data = np.expand_dims(data, axis=0)
        
        if self.transform is not None:
            data = self.transform(data)
        
        return data

In [3]:
dataset = AudioDataset(path='data/원천데이터/after')

In [4]:
dataset.__len__()
dataset_indices = list(range(dataset.__len__()))
np.random.shuffle(dataset_indices)
val_split_index = int(np.floor(0.2 * dataset.__len__()))
train_idx, val_idx = dataset_indices[val_split_index:], dataset_indices[:val_split_index]

train_sampler = SubsetRandomSampler(train_idx)
val_sampler = SubsetRandomSampler(val_idx)

In [5]:
train_loader = DataLoader(dataset=dataset, batch_size=10, sampler=train_sampler, shuffle=False)
val_loader = DataLoader(dataset=dataset, batch_size=10, sampler=val_sampler, shuffle=False)

In [6]:
class VAE(nn.Module):
    def __init__(self, input_channel=1, h_dim=128*1999, z_dim=512):
        super(VAE, self).__init__()
        
        self.encoder = nn.Sequential(
            nn.Conv1d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(3, stride=2),
            nn.Conv1d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(3, stride=2),
            nn.Conv1d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(3, stride=2),
            nn.Flatten()
        )
        
        self.fc1 = nn.Linear(h_dim, z_dim)
        self.fc2 = nn.Linear(h_dim, z_dim)
        self.fc3 = nn.Linear(z_dim, h_dim)
        
        self.decoder = nn.Sequential(
            nn.Unflatten(1, (128, 1999)),
            nn.ConvTranspose1d(128, 64, kernel_size=4, stride=2, padding=0),
            nn.ReLU(),
            nn.ConvTranspose1d(64, 32, kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            nn.ConvTranspose1d(32, 1, kernel_size=4, stride=2, padding=1),
            nn.Sigmoid()
        )
        
    def reparameterization(self, mu, log_var):
        std = torch.exp(0.5*log_var)
        eps = torch.randn_like(std)
        return mu + std*eps # return z sample
    
    def bottleneck(self, h):
        mu, log_var = self.fc1(h), self.fc2(h)
        z = self.reparameterization(mu, log_var)
        return z, mu, log_var
    
    def encode(self, x):
        h = self.encoder(x)
        z, mu, log_var = self.bottleneck(h)
        return z, mu, log_var
    
    def decode(self, z):
        z = self.fc3(z)
        return self.decoder(z)
    
    def forward(self, x):
        z, mu, log_var = self.encode(x)
        recon = self.decode(z)
        return recon, mu, log_var

vae = VAE()
if torch.cuda.is_available():
    vae.cuda()

In [7]:
vae

VAE(
  (encoder): Sequential(
    (0): Conv1d(1, 32, kernel_size=(3,), stride=(1,), padding=(1,))
    (1): ReLU()
    (2): MaxPool1d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv1d(32, 64, kernel_size=(3,), stride=(1,), padding=(1,))
    (4): ReLU()
    (5): MaxPool1d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv1d(64, 128, kernel_size=(3,), stride=(1,), padding=(1,))
    (7): ReLU()
    (8): MaxPool1d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): Flatten(start_dim=1, end_dim=-1)
  )
  (fc1): Linear(in_features=255872, out_features=512, bias=True)
  (fc2): Linear(in_features=255872, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=255872, bias=True)
  (decoder): Sequential(
    (0): Unflatten(dim=1, unflattened_size=(128, 1999))
    (1): ConvTranspose1d(128, 64, kernel_size=(4,), stride=(2,))
    (2): ReLU()
    (3): ConvTranspose1d(64, 32, kernel_size=(4,), stride=(2

In [8]:
optimizer = optim.Adam(vae.parameters())

def loss_function(recon_x, x, mu, log_var):
    KLD = -0.5*torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
    BCE = F.binary_cross_entropy(recon_x, x, reduction='sum')
#     RMSE = torch.sqrt(F.mse_loss(recon_x, x))
    return KLD + BCE

In [9]:
def train(epoch):
    vae.train()
    train_loss = 0
    for batch_idx, data in enumerate(train_loader):
        data = data.cuda()
        optimizer.zero_grad()

        recon_batch, mu, log_var = vae(data)
        loss = loss_function(recon_batch, data, mu, log_var)

        loss.backward()
        train_loss += loss.item()
        optimizer.step()

        if batch_idx % 10 == 0:
            print('Train Epoch : {} [{}/{} ({:.0f}%)]\tLoss : {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item() / len(data)
            ))
    print('====> Epoch : {} Average loss : {:.4f}'.format(epoch, train_loss / len(train_loader.dataset)))

In [10]:
def val():
    vae.eval()
    val_loss = 0
    with torch.no_grad():
        for data in val_loader:
            data = data.cuda()
            recon, mu, log_var = vae(data)

            val_loss += loss_function(recon, data, mu, log_var).item()

    val_loss /= len(val_loader.dataset)
    print('====> Validation set loss : {:.4f}'.format(val_loss))

In [11]:
for epoch in range(1, 6):
    train(epoch)
    val()

Train Epoch : 1 [0/3375 (0%)]	Loss : 12344.801563
Train Epoch : 1 [100/3375 (4%)]	Loss : 1392.949902
Train Epoch : 1 [200/3375 (7%)]	Loss : 180.935767
Train Epoch : 1 [300/3375 (11%)]	Loss : 77.985284
Train Epoch : 1 [400/3375 (15%)]	Loss : 58.062152
Train Epoch : 1 [500/3375 (19%)]	Loss : 33.391003
Train Epoch : 1 [600/3375 (22%)]	Loss : 29.787753
Train Epoch : 1 [700/3375 (26%)]	Loss : 15.560243
Train Epoch : 1 [800/3375 (30%)]	Loss : 18.313412
Train Epoch : 1 [900/3375 (33%)]	Loss : -7.998346
Train Epoch : 1 [1000/3375 (37%)]	Loss : 16.113217
Train Epoch : 1 [1100/3375 (41%)]	Loss : 38.924750
Train Epoch : 1 [1200/3375 (44%)]	Loss : 30.887170
Train Epoch : 1 [1300/3375 (48%)]	Loss : 16.540128
Train Epoch : 1 [1400/3375 (52%)]	Loss : 12.639579
Train Epoch : 1 [1500/3375 (56%)]	Loss : -26.927795
Train Epoch : 1 [1600/3375 (59%)]	Loss : 13.858942
Train Epoch : 1 [1700/3375 (63%)]	Loss : 39.454092
Train Epoch : 1 [1800/3375 (67%)]	Loss : 31.163806
Train Epoch : 1 [1900/3375 (70%)]	Loss 

In [12]:
for data in train_loader:
    data = data.cuda()
    recon, mu, log_var = vae(data)
    print(f"BCE : {F.binary_cross_entropy(recon, data)}")

BCE : -0.12197447568178177
BCE : -0.11200547963380814
BCE : -0.22624577581882477
BCE : -0.15113744139671326
BCE : -0.15516945719718933
BCE : -0.08805153518915176
BCE : -0.20756888389587402
BCE : -0.1628756821155548
BCE : -0.17230063676834106
BCE : -0.2397083193063736
BCE : -0.21684449911117554
BCE : -0.060406193137168884
BCE : -0.06894868612289429
BCE : -0.16862145066261292
BCE : -0.05064377933740616
BCE : -0.23606792092323303
BCE : -0.012674706056714058
BCE : -0.025493400171399117
BCE : -0.26208528876304626
BCE : -0.2790764570236206
BCE : -0.06324616819620132
BCE : -0.24779857695102692
BCE : -0.27145007252693176
BCE : -0.1636022925376892
BCE : -0.18162648379802704
BCE : -0.011399555951356888
BCE : -0.08966405689716339
BCE : -0.04229643940925598
BCE : -0.022096311673521996
BCE : -0.24020788073539734
BCE : -0.05985831096768379
BCE : -0.10114862769842148
BCE : -0.1596486121416092
BCE : -0.04157329350709915
BCE : -0.2771729528903961
BCE : -0.17014949023723602
BCE : -0.13503403961658478
BC